In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import os

import pandas_datareader as web   
from pandas_datareader import wb
from ipywidgets import Dropdown
from ipywidgets import interact
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize
import datetime

In this project, we compare the GDP and GDP percapita of two countries United States of America and Canada across the years 2000 to 2010. We pull the data from worldbank API for the tags of GDP and GDP Per capita respectively. Then we merge it with the nauralearth_lowres data of north america using the geopandas library. 

In [4]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
na = world[world['continent'] == 'North America']

/var/folders/0y/xjyr2vqj6cv1r6b79czv4rdw0000gn/T/ipykernel_63681/3228486775.py:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))


In [5]:
indicator = ['NY.GDP.MKTP.CD', 'NY.GDP.PCAP.KD']
country = ['US', 'CA']
start = datetime.date(year = 2000, month = 1, day = 1)
end = datetime.date(year=2010, month = 12, day = 31)
na_gdp_data = wb.download(indicator= indicator, country = country, start ='2000', end = '2010')


In [6]:
na_gdp_data = na_gdp_data.reset_index().rename(columns ={'country': 'name', 'NY.GDP.MKTP.CD': 'gdp', 'NY.GDP.PCAP.KD':'gdp_percap'})


In [7]:
na_gdp_data = na_gdp_data.replace(to_replace="United States",
           value="United States of America")

In [8]:
merged_data = world.merge(na_gdp_data, on =['name'])

merged_data= merged_data.drop(['iso_a3','gdp_md_est','pop_est'], axis = 1)


In [9]:
melted_merged_data = merged_data.melt(id_vars=['name', 'year','geometry','continent'], var_name='indicator', value_name= 'value')

In [10]:
melted_merged_data[melted_merged_data['indicator'] == 'gdp_percap']

,name,year,geometry,continent,indicator,value
22,Canada,2010,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,41156.145186
23,Canada,2009,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,40368.931509
24,Canada,2008,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,42063.240117
25,Canada,2007,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,42098.431220
26,Canada,2006,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,41646.414637
27,Canada,2005,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,40989.526824
28,Canada,2004,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,40093.807340
29,Canada,2003,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,39257.761896
30,Canada,2002,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,38912.209740
31,Canada,2001,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",North America,gdp_percap,38185.219671


In [11]:



def plot_enrollment1(df, name,indicator, year):
    fig, ax = plt.subplots(figsize=(8,8))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.1)
    norm = Normalize(vmin = min(df[(df['indicator'] == indicator)]['value']), 
                             vmax = max(df[df['indicator'] == indicator]['value']))
    
    world[world['continent'] == 'North America'].plot( ax = ax, color = 'white', edgecolor = 'black');
    
    ax = melted_merged_data[(melted_merged_data['name'] == name) & 
                                (melted_merged_data['indicator'] == indicator) & (melted_merged_data['year'] == year)].plot(ax = ax,
                                                                                     cax = cax,
                                                                                     column = 'value',
                                                                                     edgecolor = 'black',
                                                                                     lw = 1,
                                                                                     norm = norm,
                                                                                     legend = True
                                                                                     )
    value = str(melted_merged_data[(melted_merged_data['name'] == name) & 
                                (melted_merged_data['indicator'] == indicator) & (melted_merged_data['year'] == year)]['value'])
    ax.axis('off');
    ax.text(0.6, -6, name + ':' + indicator +':'+ year + ':' + value, color = 'black')
    



Following function produces an interactive plot that enables the user to vizualise using a variety of options such as country name, indicator i.e GDP or GDP Per capita and Year. The chloropleth is produced and the exact value of the data point selected will be displayed beneath the graph.

In [12]:
country_names = melted_merged_data['name'].unique()
indicators = melted_merged_data['indicator'].unique()
years = melted_merged_data['year'].unique()

@interact(name = country_names, indicator = indicators, year = years)

def make_plot_enrollment1(name=country_names, indicator = indicators, year = years):
    plot_enrollment1(melted_merged_data,name , indicator, year)
    

interactive(children=(Dropdown(description='name', options=('Canada', 'United States of America'), value='Cana…